In [1]:
! pip install git+https://github.com/deepset-ai/haystack.git@ef9e4f4467a2e265bad72b048a1a3186e40969b1

  Cloning https://github.com/deepset-ai/haystack.git (to revision ef9e4f4467a2e265bad72b048a1a3186e40969b1) to /tmp/pip-req-build-bzhhp0do
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-bzhhp0do
  Running command git rev-parse -q --verify 'sha^ef9e4f4467a2e265bad72b048a1a3186e40969b1'
  Running command git fetch -q https://github.com/deepset-ai/haystack.git ef9e4f4467a2e265bad72b048a1a3186e40969b1
  Running command git checkout -q ef9e4f4467a2e265bad72b048a1a3186e40969b1
     |████████████████████████████████| 153 kB 5.3 MB/s 
     |████████████████████████████████| 54 kB 546 kB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 79 kB 5.5 MB/s 
     |████████████████████████████████| 3.0 MB 48.0 MB/s 
     |████████████████████████████████| 373 kB 42.6 MB/s 
     |████████████████████████████████| 374 kB 37.9 MB/s 
     |████████████████████████████████| 981 kB 45.4 MB/s 
     |███████████████

In [2]:
from haystack import Finder
from haystack.database.elasticsearch import ElasticsearchDocumentStore

from haystack.retriever.elasticsearch import EmbeddingRetriever
from haystack.utils import print_answers
import pandas as pd
import requests

In [3]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [6]:
import requests
requests.get('http://localhost:9200/_cluster/health').json()

{'active_primary_shards': 1,
 'active_shards': 1,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 1}

In [5]:
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="",
                                            index="document",
                                            text_field="answer",
                                            embedding_field="question_emb",
                                            embedding_dim=768,
                                            excluded_meta_data=["question_emb"])

04/29/2022 16:07:09 - INFO - elasticsearch -   GET http://localhost:9200/ [status:200 request:0.129s]
04/29/2022 16:07:09 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.510s]


In [12]:
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="deepset/sentence_bert", gpu=False)

04/29/2022 16:11:10 - INFO - haystack.retriever.elasticsearch -   Init retriever using embeddings of model deepset/sentence_bert
04/29/2022 16:11:10 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
04/29/2022 16:11:10 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
04/29/2022 16:11:13 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
04/29/2022 16:11:14 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None


In [15]:
temp = requests.get("https://raw.githubusercontent.com/uneconomicalfairy14/samvadhini/master/model/faq.csv")
open('faq.csv', 'wb').write(temp.content)
# Get dataframe with columns "question", "answer" and some custom metadata
df = pd.read_csv("faq.csv")
# Minimal cleaning
df.fillna(value="", inplace=True)
df["question"] = df["question"].apply(lambda x: x.strip())
print(df.head())

# Get embeddings for our questions from the FAQs
questions = list(df["question"].values)
df["question_emb"] = retriever.create_embedding(texts=questions)

# Convert Dataframe to list of dicts and index them in our DocumentStore
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

                                                                          question  \
0                                          What if I need to know my correct CGPA?   
1  I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?   
2  I have cleared my backlog, but the result on ERP is not updated .It still sh...   
3                               What do I do if I have not received my grade card?   
4  My name/Father's Name/Mother's name is incorrect on grade card. How do I get...   

                                                                            answer  \
0  CGPA is seen on the grade card. The CGPA count can be verified by referring ...   
1   GPA and CGPA are not mentioned on backlog grade cards. GPA, CGPA will be hi...   
2   Grade card shows first attempt GPA and CGPA only. Hence, even if you have c...   
3  Grade cards should be collected from the student section of your Department....   
4   Name correction on grade cards/PDC needs to be do

Inferencing Samples: 100%|██████████| 23/23 [03:12<00:00,  8.35s/ Batches]
04/29/2022 16:19:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.322s]


In [17]:
finder = Finder(reader=None, retriever=retriever)
prediction = finder.get_answers_via_similar_questions(question="father name wrong on grade card", top_k_retriever=10)
print_answers(prediction, details="medium")

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.00s/ Batches]
/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchWarning)
04/29/2022 16:21:51 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:1.029s]


[   {   'answer': ' Name correction on grade cards/PDC needs to be done in ERP '
                  'at the office of student section of your Department. Kindly '
                  'contact the office of student section for the making the '
                  'required corrections in ERP. Moreover, student must check '
                  'his/her personal details from time to time in their ERP '
                  'login.',
        'context': ' Name correction on grade cards/PDC needs to be done in '
                   'ERP at the office of student section of your Department. '
                   'Kindly contact the office of student section for the '
                   'making the required corrections in ERP. Moreover, student '
                   'must check his/her personal details from time to time in '
                   'their ERP login.',
        'score': 0.6814560999999999},
    {   'answer': ' Name correction on grade cards/PDC needs to be done in ERP '
                  'at the o

In [ ]:
from haystack.pipelines import FAQPipeline

pipe = FAQPipeline(retriever=retriever)
from haystack.utils import print_answers

prediction = pipe.run(query="How is the virus spreading?", params={"Retriever": {"top_k": 10}})
print_answers(prediction, details="medium")

In [ ]:
from haystack import Pipeline
from haystack.utils import launch_es
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader


# Initialize DocumentStore and index documents
launch_es()
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(got_docs)

# Initialize Sparse retriever
es_retriever = BM25Retriever(document_store=document_store)

# Initialize dense retriever
embedding_retriever = EmbeddingRetriever(
    document_store,
    model_format="sentence_transformers",
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
)
document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

# Initialize reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

# Prebuilt pipeline
p_extractive_premade = ExtractiveQAPipeline(reader=reader, retriever=es_retriever)
res = p_extractive_premade.run(
    query="Where can i get a duplicate grade card", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(res, details="minimum")